In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
llm.invoke("hi,how are you?")

### Pre-difined Tools

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchan"}))

In [ ]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("sunny savita")

In [ ]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("carry minati")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

### Custom tools

In [ ]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [ ]:
get_word_length.invoke('Hii')

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
@tool
def summisation(a: int, b: int) -> int:
    """Adding two numbers."""
    return a + b

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

### Agents

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [ ]:
tool=load_tools(["wikipedia"],llm=llm)

In [ ]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
agent.run('who is keerthi suresh')

In [ ]:
agent.run("what is llama and who create this llm model?")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search = TavilySearchResults()

In [ ]:
search.invoke("what is the weather in SF")

In [ ]:
tools = [search]

In [ ]:
from langchain import hub

In [ ]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt

In [ ]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is the corrent GDP of India"})

In [ ]:

print(agent_executor.invoke({"input": "whats the weather in new york"}))

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [ ]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [ ]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("what is longsmit")[0]

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "hi! what is a langsmith?"})

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})

### React Agent

In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [51]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [52]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [53]:
prompt = PromptTemplate.from_template(template)

In [54]:
search_agent = create_react_agent(llm,tools,prompt)

In [55]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [59]:
agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who won the 2007 US PGA Championship and where they are from. I also need to find their score. 
Action: Intermediate Answer
Action Input: Who won the 2007 US PGA Championship?The 2007 PGA Championship was the 89th PGA Championship, played August 9–12 at Southern Hills Country Club in Tulsa, Oklahoma. Defending champion Tiger Woods won his fourth PGA Championship and 13th major title, two strokes ahead of runner-up Woody Austin.Thought: Tiger Woods won the 2007 US PGA Championship. I need to find his hometown. 
Action: Intermediate Answer
Action Input: Where is Tiger Woods from?Cypress, CAThought: I now know Tiger Woods' hometown and score. 
Final Answer: Tiger Woods, who is from Cypress, CA, won the 2007 US PGA Championship with a score of 13 under par. 


> Finished chain.


{'input': 'Where is the hometown of the 2007 US PGA championship winner and his score?',
 'output': 'Tiger Woods, who is from Cypress, CA, won the 2007 US PGA Championship with a score of 13 under par.',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='Who won the 2007 US PGA Championship?', log='Thought: I need to find out who won the 2007 US PGA Championship and where they are from. I also need to find their score. \nAction: Intermediate Answer\nAction Input: Who won the 2007 US PGA Championship?'),
   'The 2007 PGA Championship was the 89th PGA Championship, played August 9–12 at Southern Hills Country Club in Tulsa, Oklahoma. Defending champion Tiger Woods won his fourth PGA Championship and 13th major title, two strokes ahead of runner-up Woody Austin.'),
  (AgentAction(tool='Intermediate Answer', tool_input='Where is Tiger Woods from?', log='Thought: Tiger Woods won the 2007 US PGA Championship. I need to find his hometown. \nAction: Intermediate Answe

### ReAct agent with custom tools

In [60]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [61]:
# Custom tool for the Agent 
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

In [62]:
# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [63]:
# Saved React Prompt in langchain hub, we could manually type the prompt as well.
prompt = hub.pull("hwchase17/react")

In [64]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [65]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [66]:
tools = [get_employee_salary, get_employee_id]

In [67]:
agent = create_react_agent(llm,tools,prompt)

In [68]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [69]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
Thought: I need to find Evan's employee ID first, then use that to get his salary.
Action: get_employee_id
Action Input: EvanE005Thought: Now that I have Evan's employee ID, I can get his salary.
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: 45000 


> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': '45000'}

### Self Ask with serach agent

In [70]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

In [71]:
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

In [72]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")

In [73]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'self-ask-with-search', 'lc_hub_commit_hash': '9ef33d1fc82608bdfaa88b3757306ec4f22511b736cb267b1737616641dac0cc'} template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: W

In [74]:
# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [75]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [77]:
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
Yes. 


Follow up: Who is the reigning men's U.S. Open champion? The reigning men's U.S. Open champion is Novak Djokovic, who won in 2023. He defeated Daniil Medvedev in the final. Djokovic's win marks his fourth U.S. Open title.

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: Follow up: What is Novak Djokovic's hometown?

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 